# Preparation

In [1]:
use_colab = True
if use_colab:
    !pip install transformers[sentencepiece]
    from google.colab import drive
    drive.mount('/content/drive')
    path = "/content/drive/MyDrive/CAPP30255_Project/twitter_disaster_detection"


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tenso

Mounted at /content/drive


In [2]:
import os
import copy
import pandas as pd
import numpy as np
# from tqdm.autonotebook import tqdm
from tqdm import tqdm
import random
import json

import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split, KFold

# importing HuggingFace transformers library
import transformers
from transformers import pipeline, get_linear_schedule_with_warmup

print("Transformer version:", transformers.__version__)
print("torch.device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))

Transformer version: 4.29.2
torch.device: cuda


In [3]:
data = pd.read_csv(path+'/data/cleaned-train-tweets.csv', sep="|")
display(data.sample(5))


,id,keyword,location,text,target,clean_text
13246,13246,flooding,Jakarta/Kuala Lumpur/S'pore,Heavy Rainfall and Flooding in Northern #VietN...,1,heavy rainfall flooding northern vietnam situa...
16688,16688,structural%20failure,NaN,Investigators say a fatal Virgin Galactic spac...,1,investigator say fatal virgin galactic spacesh...
3745,3745,fire,NaN,My asshole is on fire https://t.co/Y3FO0gHg8t,0,asshole fire http
14069,14069,injured,Nigeria,Ogun smugglers engage Customs in shootoutåÊ: S...,1,ogun smuggler engage custom shootoutåê several...
3301,3301,evacuate,NaN,Condemnation clearly replacing the latest resp...,1,condemnation clearly replacing latest response...


# BERT without fine-tuning

In [ ]:
simple_model = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [ ]:
labels = ["not disaster", "disaster"]

def model_simple_test(i):
    responce= simple_model(
        data["text"][i],
        candidate_labels=labels
    )
    print(responce)

    true_label = labels[1] if data["target"][i] else labels[0]
    print("True Label:", true_label)
    print("Prediction:", responce["labels"][np.argmax(responce["scores"])])

def calculate_accuracy_zscppl(data, target_ids):
    accurate_num = 0
    for i in target_ids:
        response = simple_model(
            data["text"][i],
            candidate_labels=labels
        )
        true_label = labels[1] if data["target"][i] else labels[0]
        if true_label == response["labels"][np.argmax(response["scores"])]:
            accurate_num += 1
    return accurate_num / len(target_ids)


In [ ]:
data_sample = data.sample(5)
data_sample_ids = data_sample["id"].tolist()
display(data_sample)

for id in data_sample_ids:
    model_simple_test(id)

,id,keyword,location,text,target,clean_text
3782,3782,fire%20truck,District 12 - Orange County,SIGALERT UPDATE #3***N-133 CLOSED AT 5 FWY UFN...,1,sigalert update closed fwy ufn trash truck fire
9491,9491,burning,NaN,@aubilenon @MarkKriegsman if you think you'd l...,0,aubilenon markkriegsman think like burning man...
8573,8573,blaze,Delhi,#socialmedia news - New Facebook Page Features...,0,socialmedia news new facebook page feature see...
9514,9514,burning,"TÌÁchira, Venezuela",the Burning Legion has returned,0,burning legion returned
14290,14290,landslide,NaN,5 need to-dos seeing as how technical writing ...,0,need seeing technical writing administer apps ...


{'sequence': 'SIGALERT UPDATE #3***N-133 CLOSED AT 5 FWY UFN***- TRASH TRUCK FIRE', 'labels': ['disaster', 'not disaster'], 'scores': [0.9129602909088135, 0.08703970164060593]}
True Label: disaster
Prediction: disaster
{'sequence': "@aubilenon @MarkKriegsman if you think you'd like burning man you should try it because it's the only way to know!", 'labels': ['disaster', 'not disaster'], 'scores': [0.7511258721351624, 0.24887412786483765]}
True Label: not disaster
Prediction: disaster
{'sequence': '#socialmedia news - New Facebook Page Features Seek to Help Personalize the Customer Experience http://t.co/nbizaTlsmV', 'labels': ['not disaster', 'disaster'], 'scores': [0.9289625883102417, 0.07103737443685532]}
True Label: not disaster
Prediction: not disaster
{'sequence': 'the Burning Legion has returned', 'labels': ['disaster', 'not disaster'], 'scores': [0.9670848250389099, 0.0329151451587677]}
True Label: not disaster
Prediction: disaster
{'sequence': '5 need to-dos seeing as how techn

In [ ]:
num_target = 50
target_ids = [random.randint(0, len(data)) for _ in range(num_target)]
accuracy_rate = calculate_accuracy_zscppl(data, target_ids)
print(f"Accuracy Rate using Pretrained BERT Zero-Shot-Classification: {accuracy_rate}")


Accuracy Rate using Pretrained BERT Zero-Shot-Classification: 0.6


# DistilBERT model with fine-tuning

## Preparation

### Building A PyTorch Dataset

The following code uses the idea from this tutorial [Fine-tuning with custom datasets](https://huggingface.co/transformers/v3.2.0/custom_datasets.html) on building a custom dataset:


In [4]:
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, tokenizer, mode="train", max_length=None):
        self.dataframe = dataframe
        if mode != "test":
            self.targets = dataframe['target'].values
        texts = list(dataframe['text'].values)
        self.encodings = tokenizer(texts, 
                                   padding=True, 
                                   truncation=True, 
                                   max_length=max_length)
        self.mode = mode


    def __getitem__(self, idx):
        # putting each tensor in front of the corresponding key from the tokenizer
        # HuggingFace tokenizers give you whatever you need to feed to the corresponding model
        item = {key: torch.tensor(values[idx]) for key, values in self.encodings.items()}
        # when testing, there are no targets so we won't do the following
        if self.mode != "test":
            item['labels'] = torch.tensor(self.targets[idx])
        return item


    def __len__(self):
        return len(self.dataframe)


Just a wrapper to easier build the Dataset and DataLoader

In [5]:
def make_loaders(dataframe, tokenizer, mode="train", max_length=None):
    dataset = TweetDataset(dataframe, tokenizer, mode, max_length=max_length)
    dataloader = torch.utils.data.DataLoader(dataset, 
                                             batch_size=options.batch_size, 
                                             shuffle=True if mode == "train" else False,
                                             num_workers=options.num_workers)
    return dataloader

### Custom Classification Model based on DistilBERT

* DistilBERT is a Language Model which needs to be fine-tuned on a final task of interest. So, we need to build that custom head here. In the [BERT paper](https://arxiv.org/abs/1810.04805)), they introduce some special tokens named [CLS] and [SEP] which they add to the sequence which is being fed to the model. [CLS] is used at the beginning of the sequence and [SEP] tokens are used to notify the end of each part in a sequence (a sequence which is going to be fed to BERT model can be made up of two parts; e.x question and corresponding text). 
 
* In the paper they explain that they use [CLS] hidden state representation to do classification tasks for the sequence. So, in our case, we are going to the same. DistilBERT model will produce a vector of size 768 as a hidden representation for this [CLS] token and we will give it to some nn.Linear layers to do our own specific task. 

In [6]:
class CustomModel(nn.Module):
    def __init__(self,
                 bert_model,
                 num_labels, 
                 bert_hidden_dim=768, 
                 classifier_hidden_dim=768, 
                 dropout=None):

        super().__init__()
        self.bert_model = bert_model
        self.head = nn.Sequential(nn.Linear(bert_hidden_dim, classifier_hidden_dim), # Do nothing if the dropout is set to None
                                  nn.ReLU(),
                                  nn.Dropout(dropout) if dropout is not None else nn.Identity(),
                                  nn.Linear(classifier_hidden_dim, num_labels))

    def forward(self, batch):
        # feeding the input_ids and masks to the model. These are provided by our tokenizer
        output = self.bert_model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
        # obtaining the last layer hidden states of the Transformer
        last_hidden_state = output.last_hidden_state # shape: (batch_size, seq_length, bert_hidden_dim)
        # CLS token is in the beginning of the sequence. So, we grab its representation by indexing the tensor containing the hidden representations
        CLS_token_state = last_hidden_state[:, 0, :]
        # passing this representation through our custom head
        logits = self.head(CLS_token_state)
        return logits


## Training and Evaluation

In [7]:
class AvgMeter:
    def __init__(self, name="Metric"):
        self.name = name
        self.reset()
    
    def reset(self):
        self.avg, self.sum, self.count = [0]*3
    
    def update(self, val, count=1):
        self.count += count
        self.sum += val * count
        self.avg = self.sum / self.count
    
    def __repr__(self):
        text = f"{self.name}: {self.avg:.4f}"
        return text

def one_epoch(model, criterion, loader, device, optimizer=None, lr_scheduler=None, mode="train", step="batch"):
    loss_meter = AvgMeter()
    acc_meter = AvgMeter()
    
    tqdm_object = tqdm(loader, total=len(loader))
    for batch in tqdm_object:
        batch = {k: v.to(device) for k, v in batch.items()}
        preds = model(batch)
        loss = criterion(preds, batch['labels'])
        if mode == "train":
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if step == "batch":
                lr_scheduler.step()
                
        count = batch['input_ids'].size(0)
        loss_meter.update(loss.item(), count)
        
        accuracy = get_accuracy(preds.detach(), batch['labels'])
        acc_meter.update(accuracy.item(), count)
        if mode == "train":
            tqdm_object.set_postfix(loss=loss_meter.avg, accuracy=acc_meter.avg, lr=get_lr(optimizer))
        else:
            tqdm_object.set_postfix(loss=loss_meter.avg, accuracy=acc_meter.avg)
    
    return loss_meter, acc_meter

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group["lr"]

def get_accuracy(preds, targets):
    """
    preds shape: (batch_size, num_labels)
    targets shape: (batch_size)
    """
    preds = preds.argmax(dim=1)
    acc = (preds == targets).float().mean()
    return acc


In [8]:
def train_eval(epochs, model, train_loader, valid_loader, 
               criterion, optimizer, device, options, loss_accuracy, lr_scheduler=None):
    
    best_loss = float('inf')
    best_model_weights = copy.deepcopy(model.state_dict())
    
    for epoch in range(epochs):
        print("~" * 30)
        print(f"Epoch {epoch + 1}")
        current_lr = get_lr(optimizer)
        
        model.train()
        train_loss, train_acc = one_epoch(model, 
                                          criterion, 
                                          train_loader, 
                                          device,
                                          optimizer=optimizer,
                                          lr_scheduler=lr_scheduler,
                                          mode="train",
                                          step=options.step)                     
        model.eval()
        with torch.no_grad():
            valid_loss, valid_acc = one_epoch(model, 
                                              criterion, 
                                              valid_loader, 
                                              device,
                                              optimizer=None,
                                              lr_scheduler=None,
                                              mode="valid")
        
        if valid_loss.avg < best_loss:
            best_loss = valid_loss.avg
            best_model_weights = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), f'{options.model_path}/{options.model_save_name}')
            print("Saved best model!")
        
        # or you could do: if step == "epoch":
        if isinstance(lr_scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
            lr_scheduler.step(valid_loss.avg)
            # if the learning rate changes by ReduceLROnPlateau, we are going to
            # reload our previous best model weights and start from there with a lower LR
            if current_lr != get_lr(optimizer):
                print("Loading best model weights!")
                model.load_state_dict(torch.load(f'{options.model_path}/{options.model_save_name}', 
                                                 map_location=device))
        

        print(f"Train Loss: {train_loss.avg:.5f}")
        print(f"Train Accuracy: {train_acc.avg:.5f}")
        
        print(f"Valid Loss: {valid_loss.avg:.5f}")
        print(f"Valid Accuracy: {valid_acc.avg:.5f}")
        print("*" * 30)

        loss_accuracy["Train Loss"] += [train_loss.avg]
        loss_accuracy["Train Accuracy"] += [train_acc.avg]
        loss_accuracy["Valid Loss"] += [valid_loss.avg]
        loss_accuracy["Valid Accuracy"] += [valid_acc.avg]


### K-Fold Cross Validation

In [9]:
def make_folds(dataframe, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    for i, (_, valid_idx) in enumerate(kf.split(X=dataframe['id'])):
        dataframe.loc[valid_idx, 'fold'] = i
    return dataframe

In [10]:
def one_fold(fold, options):  
    print(f"Training Fold: {fold}")
    loss_accuracy = {"Train Loss": [], "Train Accuracy": [], "Valid Loss": [], "Valid Accuracy": []}
    
    # Here, we load the pre-trained DistilBERT model from transformers library
    bert_model = transformers.DistilBertModel.from_pretrained(options.model_name)
    # Loading the corresponding tokenizer from HuggingFace by using AutoTokenizer class.
    tokenizer = transformers.AutoTokenizer.from_pretrained(options.model_name, use_fast=True)
    
    dataframe = pd.read_csv(path+'/data/cleaned-train-tweets.csv', sep="|")
    dataframe = make_folds(dataframe, n_splits=options.n_folds)
    train_dataframe = dataframe[dataframe['fold'] != fold].reset_index(drop=True)
    valid_dataframe = dataframe[dataframe['fold'] == fold].reset_index(drop=True)

    train_loader = make_loaders(train_dataframe, tokenizer, "train", options.max_length)
    valid_loader = make_loaders(valid_dataframe, tokenizer, "valid", options.max_length)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = CustomModel(bert_model, options.num_labels, dropout=options.dropout).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=options.learning_rate)
    if options.scheduler == "ReduceLROnPlateau":
        lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                                  mode="min", 
                                                                  factor=0.5, 
                                                                  patience=options.patience)

        # when to step the scheduler: after an epoch or after a batch
        options.step = "epoch"
        
    elif options.scheduler == "LinearWarmup":
        num_train_steps = len(train_loader) * options.epochs
        lr_scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                       num_warmup_steps=0, 
                                                       num_training_steps=num_train_steps)
        
        # when to step the scheduler: after an epoch or after a batch
        options.step = "batch"
    
    criterion = nn.CrossEntropyLoss()
    options.model_save_name = f"model_fold_{fold}.pt"
    train_eval(options.epochs, model, train_loader, valid_loader,
               criterion, optimizer, device, options, loss_accuracy, lr_scheduler=lr_scheduler)

    # tf = open(f"{path}/models/loss_accuracy_{fold}.json", "x")
    # json.dump(loss_accuracy, tf)
    # tf.close()


In [11]:
def train_folds(options):
    n_folds = options.n_folds
    for i in range(n_folds):
        one_fold(fold=i, options=options)

## Tuning

In [ ]:
class Options:
    model_name = 'distilbert-base-uncased'
    batch_size = 32
    num_labels = 2
    epochs = 3
    num_workers = 2
    learning_rate = 3e-5
    scheduler = "ReduceLROnPlateau"
    patience = 2
    dropout = 0.5
    model_path = path + "/models/"
    max_length = 140
    model_save_name = "model.pt"
    n_folds = 3
options = Options()
train_folds(options)


Training Fold: 0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 1


100%|██████████| 193/193 [00:15<00:00, 12.20it/s, accuracy=0.861, loss=0.369]


Saved best model!
Train Loss: 0.39624
Train Accuracy: 0.83484
Valid Loss: 0.36893
Valid Accuracy: 0.86148
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


...

Saved best model!
Train Loss: 0.39955
Train Accuracy: 0.82770
Valid Loss: 0.33870
Valid Accuracy: 0.86016
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


100%|██████████| 193/193 [00:15<00:00, 12.18it/s, accuracy=0.897, loss=0.278]


Saved best model!
Train Loss: 0.26119
Train Accuracy: 0.90159
Valid Loss: 0.27786
Valid Accuracy: 0.89654
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 3


100%|██████████| 193/193 [00:15<00:00, 12.18it/s, accuracy=0.888, loss=0.311]


Train Loss: 0.15444
Train Accuracy: 0.94251
Valid Loss: 0.31078
Valid Accuracy: 0.88793
******************************


In [ ]:
class Options:
    model_name = 'distilbert-base-uncased'
    batch_size = 64
    num_labels = 2
    epochs = 3
    num_workers = 2
    learning_rate = 3e-5
    scheduler = "ReduceLROnPlateau"
    patience = 2
    dropout = 0.5
    model_path = path + "/models/"
    max_length = 140
    model_save_name = "model.pt"
    n_folds = 3
options = Options()
train_folds(options)


Training Fold: 0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 1


100%|██████████| 97/97 [00:14<00:00,  6.80it/s, accuracy=0.859, loss=0.347]


Saved best model!
Train Loss: 0.40761
Train Accuracy: 0.82485
Valid Loss: 0.34736
Valid Accuracy: 0.85905
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


...

Saved best model!
Train Loss: 0.41384
Train Accuracy: 0.82113
Valid Loss: 0.35003
Valid Accuracy: 0.85870
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


100%|██████████| 97/97 [00:14<00:00,  6.61it/s, accuracy=0.885, loss=0.309]


Saved best model!
Train Loss: 0.28655
Train Accuracy: 0.89112
Valid Loss: 0.30893
Valid Accuracy: 0.88485
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 3


100%|██████████| 97/97 [00:14<00:00,  6.69it/s, accuracy=0.899, loss=0.279]


Saved best model!
Train Loss: 0.19258
Train Accuracy: 0.93245
Valid Loss: 0.27942
Valid Accuracy: 0.89881
******************************


In [ ]:
class Options:
    model_name = 'distilbert-base-uncased'
    batch_size = 128
    num_labels = 2
    epochs = 3
    num_workers = 2
    learning_rate = 3e-5
    scheduler = "ReduceLROnPlateau"
    patience = 2
    dropout = 0.5
    model_path = path + "/models/"
    max_length = 140
    model_save_name = "model.pt"
    n_folds = 3
options = Options()
train_folds(options)


Training Fold: 0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 1


100%|██████████| 49/49 [00:14<00:00,  3.31it/s, accuracy=0.857, loss=0.359]


Saved best model!
Train Loss: 0.43871
Train Accuracy: 0.80666
Valid Loss: 0.35905
Valid Accuracy: 0.85661
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


...

Saved best model!
Train Loss: 0.44064
Train Accuracy: 0.80627
Valid Loss: 0.35106
Valid Accuracy: 0.85724
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


100%|██████████| 49/49 [00:14<00:00,  3.38it/s, accuracy=0.876, loss=0.314]


Saved best model!
Train Loss: 0.31060
Train Accuracy: 0.88154
Valid Loss: 0.31368
Valid Accuracy: 0.87559
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 3


100%|██████████| 49/49 [00:14<00:00,  3.38it/s, accuracy=0.888, loss=0.297]


Saved best model!
Train Loss: 0.23450
Train Accuracy: 0.91702
Valid Loss: 0.29747
Valid Accuracy: 0.88761
******************************


In [ ]:
class Options:
    model_name = 'distilbert-base-uncased'
    batch_size = 64
    num_labels = 2
    epochs = 3
    num_workers = 2
    learning_rate = 3e-5
    scheduler = "ReduceLROnPlateau"
    patience = 2
    dropout = 0.5
    model_path = path + "/models/"
    max_length = 140
    model_save_name = "model.pt"
    n_folds = 3

options = Options()
train_folds(options)


Training Fold: 0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 1


100%|██████████| 97/97 [00:14<00:00,  6.73it/s, accuracy=0.851, loss=0.357]


Saved best model!
Train Loss: 0.40899
Train Accuracy: 0.82639
Valid Loss: 0.35676
Valid Accuracy: 0.85060
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


...

Saved best model!
Train Loss: 0.41302
Train Accuracy: 0.81926
Valid Loss: 0.33852
Valid Accuracy: 0.86292
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


100%|██████████| 97/97 [00:14<00:00,  6.63it/s, accuracy=0.887, loss=0.307]


Saved best model!
Train Loss: 0.28063
Train Accuracy: 0.89290
Valid Loss: 0.30682
Valid Accuracy: 0.88696
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 3


100%|██████████| 97/97 [00:14<00:00,  6.67it/s, accuracy=0.901, loss=0.279]


Saved best model!
Train Loss: 0.18279
Train Accuracy: 0.93513
Valid Loss: 0.27861
Valid Accuracy: 0.90141
******************************


In [ ]:
class Options:
    model_name = 'distilbert-base-uncased'
    batch_size = 64
    num_labels = 2
    epochs = 3
    num_workers = 2
    learning_rate = 3e-5
    scheduler = "ReduceLROnPlateau"
    patience = 4
    dropout = 0.5
    model_path = path + "/models/"
    max_length = 140
    model_save_name = "model.pt"
    n_folds = 3

options = Options()
train_folds(options)


Training Fold: 0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 1


100%|██████████| 97/97 [00:14<00:00,  6.78it/s, accuracy=0.848, loss=0.373]


Saved best model!
Train Loss: 0.41296
Train Accuracy: 0.81746
Valid Loss: 0.37315
Valid Accuracy: 0.84784
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


...

Saved best model!
Train Loss: 0.41310
Train Accuracy: 0.82243
Valid Loss: 0.34871
Valid Accuracy: 0.86097
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


100%|██████████| 97/97 [00:14<00:00,  6.68it/s, accuracy=0.879, loss=0.312]


Saved best model!
Train Loss: 0.28591
Train Accuracy: 0.89217
Valid Loss: 0.31213
Valid Accuracy: 0.87900
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 3


100%|██████████| 97/97 [00:14<00:00,  6.65it/s, accuracy=0.902, loss=0.268]


Saved best model!
Train Loss: 0.19294
Train Accuracy: 0.93204
Valid Loss: 0.26834
Valid Accuracy: 0.90239
******************************


In [ ]:
class Options:
    model_name = 'distilbert-base-uncased'
    batch_size = 164
    num_labels = 2
    epochs = 3
    num_workers = 2
    learning_rate = 3e-5
    scheduler = "ReduceLROnPlateau"
    patience = 8
    dropout = 0.5
    model_path = path + "/models/"
    max_length = 140
    model_save_name = "model.pt"
    n_folds = 3

options = Options()
train_folds(options)


Training Fold: 0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 1


100%|██████████| 38/38 [00:14<00:00,  2.67it/s, accuracy=0.851, loss=0.366]


Saved best model!
Train Loss: 0.43580
Train Accuracy: 0.81267
Valid Loss: 0.36562
Valid Accuracy: 0.85125
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


...

Saved best model!
Train Loss: 0.43805
Train Accuracy: 0.81098
Valid Loss: 0.35230
Valid Accuracy: 0.85886
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


100%|██████████| 38/38 [00:13<00:00,  2.82it/s, accuracy=0.879, loss=0.314]


Saved best model!
Train Loss: 0.31595
Train Accuracy: 0.87886
Valid Loss: 0.31393
Valid Accuracy: 0.87884
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 3


100%|██████████| 38/38 [00:13<00:00,  2.79it/s, accuracy=0.89, loss=0.3]


Saved best model!
Train Loss: 0.24200
Train Accuracy: 0.91288
Valid Loss: 0.30004
Valid Accuracy: 0.89037
******************************


In [ ]:
class Options:
    model_name = 'distilbert-base-uncased'
    batch_size = 64
    num_labels = 2
    epochs = 3
    num_workers = 2
    learning_rate = 3e-4
    scheduler = "ReduceLROnPlateau"
    patience = 2
    dropout = 0.5
    model_path = path + "/models/"
    max_length = 140
    model_save_name = "model.pt"
    n_folds = 3

options = Options()
train_folds(options)


Training Fold: 0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 1


100%|██████████| 97/97 [00:14<00:00,  6.79it/s, accuracy=0.841, loss=0.377]


Saved best model!
Train Loss: 0.42777
Train Accuracy: 0.82063
Valid Loss: 0.37685
Valid Accuracy: 0.84053
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


...

Saved best model!
Train Loss: 0.44255
Train Accuracy: 0.81049
Valid Loss: 0.35021
Valid Accuracy: 0.85983
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


100%|██████████| 97/97 [00:14<00:00,  6.77it/s, accuracy=0.883, loss=0.351]


Train Loss: 0.26450
Train Accuracy: 0.90435
Valid Loss: 0.35081
Valid Accuracy: 0.88306
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 3


100%|██████████| 97/97 [00:14<00:00,  6.86it/s, accuracy=0.797, loss=0.539]


Train Loss: 0.18266
Train Accuracy: 0.94186
Valid Loss: 0.53891
Valid Accuracy: 0.79682
******************************


In [ ]:
class Options:
    model_name = 'distilbert-base-uncased'
    batch_size = 64
    num_labels = 2
    epochs = 3
    num_workers = 2
    learning_rate = 3e-5
    scheduler = "ReduceLROnPlateau"
    patience = 2
    dropout = 0.5
    model_path = path + "/models/"
    max_length = 140
    model_save_name = "model.pt"
    n_folds = 3

options = Options()
train_folds(options)


Training Fold: 0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 1


100%|██████████| 97/97 [00:14<00:00,  6.74it/s, accuracy=0.855, loss=0.354]


Saved best model!
Train Loss: 0.40745
Train Accuracy: 0.82818
Valid Loss: 0.35389
Valid Accuracy: 0.85515
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


...

Saved best model!
Train Loss: 0.41074
Train Accuracy: 0.82462
Valid Loss: 0.34211
Valid Accuracy: 0.86195
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


100%|██████████| 97/97 [00:14<00:00,  6.68it/s, accuracy=0.887, loss=0.296]


Saved best model!
Train Loss: 0.27871
Train Accuracy: 0.89713
Valid Loss: 0.29615
Valid Accuracy: 0.88728
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 3


100%|██████████| 97/97 [00:14<00:00,  6.69it/s, accuracy=0.897, loss=0.293]


Saved best model!
Train Loss: 0.18972
Train Accuracy: 0.93220
Valid Loss: 0.29288
Valid Accuracy: 0.89703
******************************


In [ ]:
class Options:
    model_name = 'distilbert-base-uncased'
    batch_size = 64
    num_labels = 2
    epochs = 3
    num_workers = 2
    learning_rate = 3e-6
    scheduler = "ReduceLROnPlateau"
    patience = 2
    dropout = 0.5
    model_path = path + "/models/"
    max_length = 140
    model_save_name = "model.pt"
    n_folds = 3

options = Options()
train_folds(options)


Training Fold: 0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 1


100%|██████████| 97/97 [00:14<00:00,  6.71it/s, accuracy=0.816, loss=0.429]


Saved best model!
Train Loss: 0.56141
Train Accuracy: 0.70905
Valid Loss: 0.42886
Valid Accuracy: 0.81634
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


...

Saved best model!
Train Loss: 0.53493
Train Accuracy: 0.75893
Valid Loss: 0.40660
Valid Accuracy: 0.82930
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


100%|██████████| 97/97 [00:14<00:00,  6.69it/s, accuracy=0.847, loss=0.374]


Saved best model!
Train Loss: 0.39690
Train Accuracy: 0.83647
Valid Loss: 0.37408
Valid Accuracy: 0.84733
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 3


100%|██████████| 97/97 [00:14<00:00,  6.69it/s, accuracy=0.854, loss=0.359]


Saved best model!
Train Loss: 0.36112
Train Accuracy: 0.85442
Valid Loss: 0.35892
Valid Accuracy: 0.85399
******************************


In [ ]:
class Options:
    model_name = 'distilbert-base-uncased'
    batch_size = 64
    num_labels = 2
    epochs = 3
    num_workers = 2
    learning_rate = 3e-5
    scheduler = "ReduceLROnPlateau"
    patience = 2
    dropout = 0.3
    model_path = path + "/models/"
    max_length = 140
    model_save_name = "model.pt"
    n_folds = 3

options = Options()
train_folds(options)


Training Fold: 0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 1


100%|██████████| 97/97 [00:14<00:00,  6.73it/s, accuracy=0.859, loss=0.35]


Saved best model!
Train Loss: 0.40665
Train Accuracy: 0.82875
Valid Loss: 0.35020
Valid Accuracy: 0.85856
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


...

100%|██████████| 97/97 [00:14<00:00,  6.63it/s, accuracy=0.867, loss=0.336]


Saved best model!
Train Loss: 0.40642
Train Accuracy: 0.82421
Valid Loss: 0.33584
Valid Accuracy: 0.86731
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


100%|██████████| 97/97 [00:14<00:00,  6.70it/s, accuracy=0.889, loss=0.299]


Saved best model!
Train Loss: 0.27859
Train Accuracy: 0.89510
Valid Loss: 0.29923
Valid Accuracy: 0.88939
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 3


 58%|█████▊    | 111/193 [00:46<00:35,  2.34it/s, accuracy=0.932, loss=0.191, lr=3e-5]

In [12]:
class Options:
    model_name = 'distilbert-base-uncased'
    batch_size = 64
    num_labels = 2
    epochs = 3
    num_workers = 2
    learning_rate = 3e-5
    scheduler = "ReduceLROnPlateau"
    patience = 2
    dropout = 0.5
    model_path = path + "/models/"
    max_length = 140
    model_save_name = "model.pt"
    n_folds = 3

options = Options()
train_folds(options)


Training Fold: 0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 1


100%|██████████| 97/97 [00:15<00:00,  6.28it/s, accuracy=0.859, loss=0.349]


Saved best model!
Train Loss: 0.40448
Train Accuracy: 0.82907
Valid Loss: 0.34852
Valid Accuracy: 0.85872
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


...

Saved best model!
Train Loss: 0.41490
Train Accuracy: 0.82194
Valid Loss: 0.34152
Valid Accuracy: 0.86211
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


100%|██████████| 97/97 [00:15<00:00,  6.44it/s, accuracy=0.884, loss=0.304]


Saved best model!
Train Loss: 0.28883
Train Accuracy: 0.88982
Valid Loss: 0.30353
Valid Accuracy: 0.88420
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 3


100%|██████████| 97/97 [00:15<00:00,  6.43it/s, accuracy=0.897, loss=0.287]


Saved best model!
Train Loss: 0.19210
Train Accuracy: 0.93090
Valid Loss: 0.28713
Valid Accuracy: 0.89687
******************************


In [13]:
class Options:
    model_name = 'distilbert-base-uncased'
    batch_size = 64
    num_labels = 2
    epochs = 3
    num_workers = 2
    learning_rate = 3e-5
    scheduler = "ReduceLROnPlateau"
    patience = 2
    dropout = 0.7
    model_path = path + "/models/"
    max_length = 140
    model_save_name = "model.pt"
    n_folds = 3

options = Options()
train_folds(options)


Training Fold: 0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 1


100%|██████████| 97/97 [00:14<00:00,  6.49it/s, accuracy=0.855, loss=0.36]


Saved best model!
Train Loss: 0.42373
Train Accuracy: 0.81989
Valid Loss: 0.35962
Valid Accuracy: 0.85466
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


...

Saved best model!
Train Loss: 0.42011
Train Accuracy: 0.82243
Valid Loss: 0.34216
Valid Accuracy: 0.86682
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


100%|██████████| 97/97 [00:15<00:00,  6.36it/s, accuracy=0.882, loss=0.304]


Saved best model!
Train Loss: 0.28724
Train Accuracy: 0.89063
Valid Loss: 0.30376
Valid Accuracy: 0.88176
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 3


100%|██████████| 97/97 [00:15<00:00,  6.42it/s, accuracy=0.895, loss=0.287]


Saved best model!
Train Loss: 0.19470
Train Accuracy: 0.93196
Valid Loss: 0.28740
Valid Accuracy: 0.89524
******************************


In [14]:
class Options:
    model_name = 'distilbert-base-uncased'
    batch_size = 64
    num_labels = 2
    epochs = 3
    num_workers = 2
    learning_rate = 3e-5
    scheduler = "ReduceLROnPlateau"
    patience = 2
    dropout = 0.5
    model_path = path + "/models/"
    max_length = 140
    model_save_name = "model.pt"
    n_folds = 3

options = Options()
train_folds(options)


Training Fold: 0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 1


100%|██████████| 97/97 [00:14<00:00,  6.59it/s, accuracy=0.855, loss=0.349]


Saved best model!
Train Loss: 0.41570
Train Accuracy: 0.81876
Valid Loss: 0.34924
Valid Accuracy: 0.85466
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


...

Saved best model!
Train Loss: 0.41259
Train Accuracy: 0.82137
Valid Loss: 0.34394
Valid Accuracy: 0.86844
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


100%|██████████| 97/97 [00:15<00:00,  6.39it/s, accuracy=0.889, loss=0.296]


Saved best model!
Train Loss: 0.28152
Train Accuracy: 0.89169
Valid Loss: 0.29590
Valid Accuracy: 0.88939
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 3


100%|██████████| 97/97 [00:15<00:00,  6.44it/s, accuracy=0.901, loss=0.303]


Train Loss: 0.18581
Train Accuracy: 0.93456
Valid Loss: 0.30261
Valid Accuracy: 0.90109
******************************


In [ ]:
class Options:
    model_name = 'distilbert-base-uncased'
    batch_size = 64
    num_labels = 2
    epochs = 3
    num_workers = 2
    learning_rate = 3e-5
    scheduler = "ReduceLROnPlateau"
    patience = 2
    dropout = 0.5
    model_path = path + "/models/"
    max_length = 140
    model_save_name = "model.pt"
    n_folds = 5

options = Options()
train_folds(options)


Training Fold: 0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 1


100%|██████████| 58/58 [00:08<00:00,  6.48it/s, accuracy=0.868, loss=0.339]


Saved best model!
Train Loss: 0.40256
Train Accuracy: 0.82589
Valid Loss: 0.33863
Valid Accuracy: 0.86766
******************************
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2


 66%|██████▌   | 153/231 [01:05<00:33,  2.36it/s, accuracy=0.897, loss=0.275, lr=3e-5]

In [ ]:
class Options:
    model_name = 'distilbert-base-uncased'
    batch_size = 64
    num_labels = 2
    epochs = 3
    num_workers = 2
    learning_rate = 3e-5
    scheduler = "ReduceLROnPlateau"
    patience = 2
    dropout = 0.5
    model_path = path + "/models/"
    max_length = 140
    model_save_name = "model.pt"
    n_folds = 7

options = Options()
train_folds(options)


## Run with the best setting

In [ ]:
class Options:
    model_name = 'distilbert-base-uncased'
    batch_size = 64
    num_labels = 2
    epochs = 5
    num_workers = 2
    learning_rate = 3e-5
    scheduler = "ReduceLROnPlateau"
    patience = 2
    dropout = 0.5
    model_path = path + "/models/"
    max_length = 140
    model_save_name = "model.pt"
    n_folds = 5

options = Options()
train_folds(options)

In [ ]:
accuracies = [0.8657645459910691, 0.8960757774493691, 0.9096075771464708, 0.9098782130765334, 0.9234100128058974, 0.9142083890221923, 0.9174560208927149, 0.9139377529953426, 0.9209742888546119, 0.9179972929141518, 0.9220568328006987, 0.9198917448762628, 0.9169147489358027, 0.9182679289087391, 0.9161028409520409, 0.9155615689628663, 0.9179972929141518, 0.9188092008979136, 0.9147496609791045, 0.9150202969736917]
import matplotlib.pyplot as plt
epochs = 20
plt.plot(range(1, epochs + 1), accuracies)

In [ ]:
!nvidia-smi

In [ ]:
def test_one_model(options):  
    test_dataframe = pd.read_csv(path+'/data/tweet_samples_100.csv', sep=",")

    bert_model = transformers.DistilBertModel.from_pretrained(options.model_name)
    tokenizer = transformers.AutoTokenizer.from_pretrained(options.model_name, use_fast=True)
    
    test_loader = make_loaders(test_dataframe, tokenizer, mode="test")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = CustomModel(bert_model,
                        options.num_labels,
                        dropout=options.dropout).to(device)
    model.load_state_dict(torch.load(f"{options.model_path}/{options.model_save_name}", 
                                     map_location=device))
    model.eval()
    
    all_preds = None
    with torch.no_grad():
        for batch in tqdm(test_loader):
            batch = {k: v.to(device) for k, v in batch.items()}
            preds = model(batch)
            if all_preds is None:
                all_preds = preds
            else:
                all_preds = torch.cat([all_preds, preds], dim=0)

    return all_preds

In [ ]:
def test_all_models(options):
    n_folds = options.n_folds
    all_model_preds = []
    for fold in range(n_folds):
        options.model_save_name = f"model_fold_{fold}.pt"
        all_preds = test_one_model(options)
        all_model_preds.append(all_preds)
    
    all_model_preds = torch.stack(all_model_preds, dim=0)
    print(all_model_preds.shape)
    # I will return the mean of the final predictions of all the models
    # You could do other things like 'voting' between the five models
    return all_model_preds.mean(0)

In [ ]:
test_dataframe = pd.read_csv(path+'/data/tweet_samples_100.csv', sep=",")
all_preds = test_all_models(options)
predictions = all_preds.argmax(dim=1).cpu().numpy()
true_labels = np.array(test_dataframe["target"].tolist())

In [ ]:
np.sum(predictions == true_labels)

In [ ]:
mismatched_indices = []
for i in range(len(predictions)):
    if predictions[i] != true_labels[i]:
        mismatched_indices.append(i)
mismatched_indices